# Visualiesierung eines Neuronalen Nertzwerkes

Module Importieren:


In [1]:
import cv2, os
import pandas as pd
import numpy as np
import matplotlib.image as mpimg
import glob
from PIL import Image
from keras.models import load_model
import utils
from IPython.display import Image
import time
import seaborn as sns

Using TensorFlow backend.


Bildgrösse Festlegen:


In [2]:
IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS = 66, 200, 3
INPUT_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)

Model laden:
    

In [3]:
model = load_model('Data/Model/model.h5')
print("Model bereit")

Trainingsdaten Laden:

In [4]:
data_df = pd.read_csv(os.path.join(os.getcwd(), 'Data/Log', 'driving_log.csv'), names=['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed'])

X = data_df['center'].values
y = data_df['steering'].values

Array der Bilder

In [5]:
print(X)

['C:\\Users\\Tron\\Desktop\\Herbie\\Data\\Image\\center_2018_05_23_17_49_46_341.jpg'
 'C:\\Users\\Tron\\Desktop\\Herbie\\Data\\Image\\center_2018_05_23_17_49_46_497.jpg'
 'C:\\Users\\Tron\\Desktop\\Herbie\\Data\\Image\\center_2018_05_23_17_49_46_650.jpg'
 ...
 'C:\\Users\\Tron\\Desktop\\Herbie\\Data\\Image\\center_2018_05_22_17_05_06_175.jpg'
 'C:\\Users\\Tron\\Desktop\\Herbie\\Data\\Image\\center_2018_05_22_17_05_06_319.jpg'
 'C:\\Users\\Tron\\Desktop\\Herbie\\Data\\Image\\center_2018_05_22_17_05_06_467.jpg']


Sequenze der Steeringwerte

In [6]:
print(y)

[ 0.  0.  0. ... -1.  0. -1.]


Batchsize der Trainingsdaten

In [7]:
print("Schärfe des Image Arrays:     " + str(X.shape))
print("Schärfe der Steering Sequenz: " + str(y.shape))

Schärfe des Image Arrays:     (5180,)
Schärfe der Steering Sequenz: (5180,)


Overlay mit Orginal und Predict Werten erzeugen und als Video speichern

In [8]:
from matplotlib.pyplot import imshow

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('test.mp4', fourcc, 6.0, (360, 180))
i=0
L = list()
shape=y.shape[0]
#print(shape)
while i <= (shape - 1):
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (0,160)
    fontScale              = 1
    fontColor              = (0,255,0)
    lineType               = 2
    
    d = X[i]    
    e = y[i]
    
    img = cv2.imread(d)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    __image1 = utils.preprocess(img)
    __image2 = np.asarray(__image1, dtype=np.float32)
    __image = np.array([__image2]) 
    __steering_angle = float(model.predict(__image, batch_size=1))
    L.append(__steering_angle)
    __steering_angle = round(__steering_angle, 2)
    if i <= 2:
        imshow(__image1)
    cv2.putText(img,str(e), 
        bottomLeftCornerOfText, 
        font, 
        fontScale,
        fontColor,
        lineType)

    e = int(e * 120) 

    cv2.line(img,(160 + e,80),(160,160),(0,255,0),2)

    fontColor              = (0,0,255)
    bottomLeftCornerOfText = (0,120)

    cv2.putText(img,str(__steering_angle), 
        bottomLeftCornerOfText, 
        font, 
        fontScale,
        fontColor,
        lineType)

    __steering_angle = int(__steering_angle * 120)
    cv2.line(img,(160 + __steering_angle,80),(160,160),(0,0,255),2)

    out.write(img)
    #cv2.imwrite("Data/Imagetext/"+ str(i) + ".jpg", img)
    #cv2.imshow('image',img)
    #cv2.waitKey(0)
    
    i = i+1
cv2.destroyAllWindows
s = np.array(L)
out.release()
#print(s)

NameError: name 'model' is not defined

Video Anzeigen

In [ ]:
%%HTML
<video width="640" height="480" controls>
  <source src="test.mp4" type="video/mp4">
</video>

In [ ]:
df = pd.DataFrame({'predicted':s, 'steering':y})
ax = df.plot.scatter('predicted', 'steering')


In [ ]:
ax = pd.DataFrame({'predicted':s[:150], 'steering':y[:150]}).plot()
ax.set_ylabel("steering angle")

In [ ]:
df.diff().hist(bins=50)

In [ ]:
df.plot.hist(stacked=False, bins=30)

In [ ]:
from pandas.plotting import lag_plot
lag_plot(df)

In [ ]:
df.plot.area(stacked=False);

In [ ]:
import seaborn as sns

In [ ]:
sns.set(color_codes=True)

In [ ]:
sns.regplot(x="predicted", y="steering", data=df);

In [ ]:
sns.lmplot(x="predicted", y="steering", data=df);

In [ ]:
sns.jointplot(x="predicted", y="steering", data=df, kind="reg");